# Modeling

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Dense, Dropout, Normalization, BatchNormalization, LayerNormalization, Input
from tcn import TCN, tcn_full_summary
from catboost import CatBoostRegressor


from sklearn.preprocessing import MinMaxScaler

## Prepare the Data Sets
We want to create two separate datasets; one for our CatBoost model and one for our Keras models.

In [2]:
df = pd.read_parquet("../data/raw/input_dataset-2.parquet")
df = df.dropna()
df

,Unit_4_Power,Unit_4_Reactive Power,Turbine_Guide Vane Opening,Turbine_Pressure Drafttube,Turbine_Pressure Spiral Casing,Turbine_Rotational Speed,mode,Bolt_1_Steel tmp,Bolt_1_Tensile,Bolt_2_Tensile,...,Bolt_5_Tensile,Bolt_6_Tensile,Bolt_1_Torsion,Bolt_2_Torsion,Bolt_3_Torsion,Bolt_4_Torsion,Bolt_5_Torsion,Bolt_6_Torsion,lower_bearing_vib_vrt,turbine_bearing_vib_vrt
timepoints,,,,,,,,,,,,,,,,,,,,,
1970-12-29 10:59:57,311.093257,4.949223,94.206187,150.827828,5305.873472,108.033198,operation,2.458908,1609.140569,1488.522739,...,1637.373796,1678.107726,177.730733,163.324591,146.500052,226.013417,298.403158,161.914265,0.154292,0.517703
1970-12-29 10:59:58,311.103996,5.051777,94.206457,150.774664,5305.690188,108.033197,operation,2.458729,1609.127944,1488.494639,...,1637.353554,1678.100380,177.736039,163.323607,146.498639,226.013106,298.403272,161.912569,0.155838,0.530280
1970-12-29 10:59:59,311.114735,5.154330,94.206726,150.559452,5305.466701,108.033196,operation,2.459334,1609.138758,1488.493451,...,1637.360655,1678.106863,177.735616,163.328645,146.494156,226.013736,298.397846,161.912716,0.169547,0.540085
1970-12-29 11:00:00,311.125475,5.256883,94.206995,150.344239,5305.243213,108.033195,operation,2.459329,1609.121478,1488.488279,...,1637.365372,1678.078172,177.738552,163.331201,146.491341,226.017247,298.400658,161.912572,0.173561,0.526994
1970-12-29 11:00:01,311.136214,5.359436,94.207264,150.129027,5305.019725,108.033194,operation,2.459337,1609.113766,1488.488892,...,1637.354723,1678.067193,177.741082,163.329769,146.498284,226.013418,298.402517,161.915946,0.166955,0.524617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971-01-25 11:06:44,308.716025,3.974309,94.425666,157.927905,5280.929965,108.057498,operation,4.193937,1637.386115,1504.557822,...,1640.704071,1690.014705,183.204777,178.082932,145.759475,225.351989,300.667011,160.949816,0.159366,0.491265
1971-01-25 11:06:45,308.746393,4.103262,94.429003,157.974925,5280.633358,108.057492,operation,4.193254,1637.365865,1504.546091,...,1640.711250,1690.017029,183.203293,178.082287,145.766584,225.348279,300.674243,160.947868,0.155962,0.497242
1971-01-25 11:06:46,308.776762,4.472929,94.432340,158.021945,5280.336751,108.057486,operation,4.193261,1637.384133,1504.538696,...,1640.699142,1690.002008,183.212397,178.081678,145.764007,225.354785,300.674078,160.947644,0.141150,0.501525


In [3]:
X_cols = ["Unit_4_Power", "Unit_4_Reactive Power", "Turbine_Guide Vane Opening", "Turbine_Pressure Drafttube", "Turbine_Pressure Spiral Casing", "Turbine_Rotational Speed"]
y_cols = [c for c in df if c.endswith("Tensile")]

### Create CatBoost dataset

In [37]:
lookback = 30

cX = df[X_cols].copy()
for i in range(1, lookback+1):
    cX.loc[:, [f"{c} (t-{i})" for c in X_cols]] = cX[X_cols].shift(i).rename(columns={c: f"{c} (t-{i})" for c in X_cols})

cy = df[y_cols]

cX

,Unit_4_Power,Unit_4_Reactive Power,Turbine_Guide Vane Opening,Turbine_Pressure Drafttube,Turbine_Pressure Spiral Casing,Turbine_Rotational Speed,Unit_4_Power (t-1),Unit_4_Reactive Power (t-1),Turbine_Guide Vane Opening (t-1),Turbine_Pressure Drafttube (t-1),...,Turbine_Guide Vane Opening (t-29),Turbine_Pressure Drafttube (t-29),Turbine_Pressure Spiral Casing (t-29),Turbine_Rotational Speed (t-29),Unit_4_Power (t-30),Unit_4_Reactive Power (t-30),Turbine_Guide Vane Opening (t-30),Turbine_Pressure Drafttube (t-30),Turbine_Pressure Spiral Casing (t-30),Turbine_Rotational Speed (t-30)
timepoints,,,,,,,,,,,,,,,,,,,,,
1970-12-29 10:59:57,311.093257,4.949223,94.206187,150.827828,5305.873472,108.033198,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-12-29 10:59:58,311.103996,5.051777,94.206457,150.774664,5305.690188,108.033197,311.093257,4.949223,94.206187,150.827828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-12-29 10:59:59,311.114735,5.154330,94.206726,150.559452,5305.466701,108.033196,311.103996,5.051777,94.206457,150.774664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-12-29 11:00:00,311.125475,5.256883,94.206995,150.344239,5305.243213,108.033195,311.114735,5.154330,94.206726,150.559452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-12-29 11:00:01,311.136214,5.359436,94.207264,150.129027,5305.019725,108.033194,311.125475,5.256883,94.206995,150.344239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971-01-25 11:06:44,308.716025,3.974309,94.425666,157.927905,5280.929965,108.057498,308.685656,4.382225,94.422329,157.885326,...,94.328895,158.429536,5281.981009,108.057682,307.804965,5.665293,94.325558,158.381809,5281.938473,108.057688
1971-01-25 11:06:45,308.746393,4.103262,94.429003,157.974925,5280.633358,108.057492,308.716025,3.974309,94.425666,157.927905,...,94.332232,158.477262,5282.023546,108.057675,307.835334,5.529349,94.328895,158.429536,5281.981009,108.057682
1971-01-25 11:06:46,308.776762,4.472929,94.432340,158.021945,5280.336751,108.057486,308.746393,4.103262,94.429003,157.974925,...,94.335569,158.524989,5282.066082,108.057669,307.865702,5.393404,94.332232,158.477262,5282.023546,108.057675


In [5]:
def train_test_split(X, y, test_percent=0.1, offset_percent=0):
    
    test_start = int(len(df) * offset_percent)
    test_end = int(len(df) * (offset_percent + test_percent))

    train = df.iloc[:test_start]
    test = df.iloc[test_start:test_end]

    X_train, X_test = X.iloc[:test_start], X.iloc[test_start:test_end]
    y_train, y_test = y.iloc[:test_start], y.iloc[test_start:test_end]
    
    return X_train, X_test, y_train, y_test

cX_train, cX_test, cy_train, cy_test = train_test_split(cX, cy, test_percent=0.1, offset_percent=0.9)

### CatBoost Modeling

In [6]:
def train_catboost(X_train, y_train, eval_set=None, params={}):
    
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=eval_set)
    
    return model

In [7]:
def plot_error(X_test, y_test, model):

    y_test = y_tests[j].copy()

    pred = model.predict(X_test)

    plt.subplots(figsize=(15, 10))
    plt.scatter(y_test.index, y_test, label="real", s=2)
    plt.scatter(y_test.index, pred, label="pred", s=2)
    plt.legend()
    plt.show()

In [38]:
def cv_catboost(X, y, n=4, start_offset=0.5, verbose=False, params={}):

    test_percent = (1 - start_offset) / n

    all_results = []
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            test_percent = test_percent,
                                                            offset_percent = start_offset + i*test_percent)
        y_trains = [y_train[c] for c in y_train]
        y_tests = [y_test[c] for c in y_test]

        results = []
        for j in range(len(y_trains)):
            model = CatBoostRegressor(**params)
            model.fit(X_train, y_trains[j], eval_set=(X_test, y_tests[j]), verbose=verbose)

            pred = model.predict(X_test)
            mape = 100 * ((y_tests[j] - pred).abs() / y_tests[j]).mean()
            results.append(mape)
            print(f"iteration {i}, bolt {j}: MAPE={mape}")
        all_results.append(results)
    all_results = np.array(all_results)
    
    return all_results

params = {
    "loss_function": "MAPE",
    "iterations": 50,
    "depth": 5
}

results = cv_catboost(cX, cy, params=params)

iteration 0, bolt 0: MAPE=0.12893589693539015
iteration 0, bolt 1: MAPE=0.04987014868448909
iteration 0, bolt 2: MAPE=0.02270766569173188
iteration 0, bolt 3: MAPE=0.03212724265100928
iteration 0, bolt 4: MAPE=0.0290831762786969
iteration 0, bolt 5: MAPE=0.036503443837256525
iteration 1, bolt 0: MAPE=0.4045819267500691
iteration 1, bolt 1: MAPE=0.25709853272160343
iteration 1, bolt 2: MAPE=0.1612081811439825
iteration 1, bolt 3: MAPE=0.11544980708691395
iteration 1, bolt 4: MAPE=0.10593890252012066
iteration 1, bolt 5: MAPE=0.13675306513130692
iteration 2, bolt 0: MAPE=0.6743007702934756
iteration 2, bolt 1: MAPE=0.5206425236108078
iteration 2, bolt 2: MAPE=0.25521718411069616
iteration 2, bolt 3: MAPE=0.14367768594093294
iteration 2, bolt 4: MAPE=0.16331549583706603
iteration 2, bolt 5: MAPE=0.3082450966613017
iteration 3, bolt 0: MAPE=0.7879159894932666
iteration 3, bolt 1: MAPE=0.4406906427150564
iteration 3, bolt 2: MAPE=0.28231155132687785
iteration 3, bolt 3: MAPE=0.1277020284551

In [53]:
pd.DataFrame(results).mean(axis=1)

0    0.049871
1    0.196838
2    0.344233
3    0.340283
dtype: float64

In [39]:
results.mean()

0.23280644419053212

In [55]:
X_train, X_test, y_train, y_test = train_test_split(cX,
                                                    cy,
                                                    test_percent = 0.125,
                                                    offset_percent = 0.5 + 3*0.125)

y_trains = [y_train[c] for c in y_train]
y_tests = [y_test[c] for c in y_test]

model = CatBoostRegressor(**params)
model.fit(X_train, y_trains[0], eval_set=(X_test, y_tests[0]), verbose=True)

0:	learn: 0.0027983	test: 0.0103647	best: 0.0103647 (0)	total: 154ms	remaining: 7.55s
1:	learn: 0.0027540	test: 0.0102832	best: 0.0102832 (1)	total: 297ms	remaining: 7.14s
2:	learn: 0.0027104	test: 0.0101959	best: 0.0101959 (2)	total: 444ms	remaining: 6.96s
3:	learn: 0.0026682	test: 0.0101021	best: 0.0101021 (3)	total: 591ms	remaining: 6.79s
4:	learn: 0.0026275	test: 0.0100109	best: 0.0100109 (4)	total: 730ms	remaining: 6.57s
5:	learn: 0.0025884	test: 0.0099304	best: 0.0099304 (5)	total: 861ms	remaining: 6.32s
6:	learn: 0.0025504	test: 0.0098423	best: 0.0098423 (6)	total: 992ms	remaining: 6.1s
7:	learn: 0.0025139	test: 0.0097653	best: 0.0097653 (7)	total: 1.12s	remaining: 5.9s
8:	learn: 0.0024784	test: 0.0096823	best: 0.0096823 (8)	total: 1.27s	remaining: 5.77s
9:	learn: 0.0024442	test: 0.0096093	best: 0.0096093 (9)	total: 1.4s	remaining: 5.61s
10:	learn: 0.0024111	test: 0.0095387	best: 0.0095387 (10)	total: 1.54s	remaining: 5.46s
11:	learn: 0.0023791	test: 0.0094700	best: 0.0094700 (1

In [58]:
pred = model.predict(X_test)
((y_tests[j] - pred).abs() / y_tests[j]).mean()

0.007879159894932666

In [50]:
y_tests[j] - pred

,Bolt_1_Tensile,Bolt_2_Tensile,Bolt_3_Tensile,Bolt_4_Tensile,Bolt_5_Tensile,Bolt_6_Tensile
timepoints,,,,,,
1970-12-29 10:59:57,1609.140569,1488.522739,1688.519258,1603.236155,1637.373796,1678.107726
1970-12-29 10:59:58,1609.127944,1488.494639,1688.513150,1603.235101,1637.353554,1678.100380
1970-12-29 10:59:59,1609.138758,1488.493451,1688.514689,1603.236408,1637.360655,1678.106863
1970-12-29 11:00:00,1609.121478,1488.488279,1688.518252,1603.230738,1637.365372,1678.078172
1970-12-29 11:00:01,1609.113766,1488.488892,1688.517709,1603.246668,1637.354723,1678.067193
...,...,...,...,...,...,...
1971-01-22 09:20:55,1634.998080,1503.554290,1699.357288,1606.688402,1640.698024,1689.009182
1971-01-22 09:20:56,1635.004269,1503.564389,1699.355708,1606.686139,1640.702035,1689.004366
1971-01-22 09:20:57,1635.020589,1503.579058,1699.364125,1606.694313,1640.725198,1689.007927


In [49]:
y_test

,Bolt_1_Tensile,Bolt_2_Tensile,Bolt_3_Tensile,Bolt_4_Tensile,Bolt_5_Tensile,Bolt_6_Tensile
timepoints,,,,,,
1971-01-22 09:21:00,1635.019961,1503.588666,1699.390838,1606.709155,1640.770551,1689.029392
1971-01-22 09:21:01,1635.025671,1503.599601,1699.389898,1606.712710,1640.797633,1689.038937
1971-01-22 09:21:02,1635.039120,1503.622544,1699.371938,1606.721674,1640.811085,1689.061857
1971-01-22 09:21:03,1635.032899,1503.602246,1699.378993,1606.736343,1640.811581,1689.049959
1971-01-22 09:21:04,1635.021613,1503.596760,1699.381459,1606.727942,1640.823612,1689.069089
...,...,...,...,...,...,...
1971-01-25 11:06:44,1637.386115,1504.557822,1701.651420,1606.276545,1640.704071,1690.014705
1971-01-25 11:06:45,1637.365865,1504.546091,1701.654301,1606.271877,1640.711250,1690.017029
1971-01-25 11:06:46,1637.384133,1504.538696,1701.656143,1606.250028,1640.699142,1690.002008


## Create Keras datasets

In [9]:
test_size = 1000

kX = df[X_cols].copy()
ky = df[y_cols]

kX_train_df, kX_test_df, ky_train_df, ky_test_df = train_test_split(kX, ky, test_percent=0.1, offset_percent=0.9)
kX_train_df

,Unit_4_Power,Unit_4_Reactive Power,Turbine_Guide Vane Opening,Turbine_Pressure Drafttube,Turbine_Pressure Spiral Casing,Turbine_Rotational Speed
timepoints,,,,,,
1970-12-29 10:59:57,311.093257,4.949223,94.206187,150.827828,5305.873472,108.033198
1970-12-29 10:59:58,311.103996,5.051777,94.206457,150.774664,5305.690188,108.033197
1970-12-29 10:59:59,311.114735,5.154330,94.206726,150.559452,5305.466701,108.033196
1970-12-29 11:00:00,311.125475,5.256883,94.206995,150.344239,5305.243213,108.033195
1970-12-29 11:00:01,311.136214,5.359436,94.207264,150.129027,5305.019725,108.033194
...,...,...,...,...,...,...
1971-01-22 15:54:13,310.908386,-0.772782,94.888753,155.644087,5297.637203,108.037498
1971-01-22 15:54:14,310.908386,-1.020309,94.888718,155.853108,5297.777458,108.037498
1971-01-22 15:54:15,310.908386,-1.267836,94.888682,156.062128,5297.917714,108.037498


In [10]:
def lstm_data_transform(x_data, y_data, num_steps=5):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

ky_trains = [ky_train_df[c] for c in ky_train_df]
ky_tests = [ky_test_df[c] for c in ky_test_df]

j = 0
kX_train, ky_train = lstm_data_transform(kX_train_df, ky_trains[j], num_steps=30)
kX_test, ky_test = lstm_data_transform(kX_test_df, ky_tests[j], num_steps=30)

In [11]:
kX_train.shape

(849507, 30, 6)

In [12]:
kX_train

array([[[ 3.11093257e+02,  4.94922343e+00,  9.42061873e+01,
          1.50827828e+02,  5.30587347e+03,  1.08033198e+02],
        [ 3.11103996e+02,  5.05177662e+00,  9.42064566e+01,
          1.50774664e+02,  5.30569019e+03,  1.08033197e+02],
        [ 3.11114735e+02,  5.15432981e+00,  9.42067258e+01,
          1.50559452e+02,  5.30546670e+03,  1.08033196e+02],
        ...,
        [ 3.11383219e+02,  5.91532551e+00,  9.42134576e+01,
          1.49317819e+02,  5.30457285e+03,  1.08033174e+02],
        [ 3.11393958e+02,  5.93561389e+00,  9.42137269e+01,
          1.49295828e+02,  5.30459147e+03,  1.08033173e+02],
        [ 3.11404697e+02,  5.95590227e+00,  9.42139961e+01,
          1.49273837e+02,  5.30461010e+03,  1.08033173e+02]],

       [[ 3.11103996e+02,  5.05177662e+00,  9.42064566e+01,
          1.50774664e+02,  5.30569019e+03,  1.08033197e+02],
        [ 3.11114735e+02,  5.15432981e+00,  9.42067258e+01,
          1.50559452e+02,  5.30546670e+03,  1.08033196e+02],
        [ 3.11125

In [22]:
def plot_error(model, X_test, y_test):
    pred = pd.DataFrame(model.predict(X_test)).iloc[:, 0]
    pd.concat([pred, pd.DataFrame(y_test)], axis=1).plot()

In [23]:
def get_mse(model, X_test, y_test, scaler=None):
    pred = model.predict(X_test)
    if scaler is not None:
        pred = scaler.inverse_transform(pred)
        y_test = scaler.inverse_transform(y_test)
    return ((pred - y_test)**2).mean()

## Train Keras models

In [29]:
def train_keras(first, X_train, y_train, eval_set, epochs=20, batch_size=2048):
    
    norm = Normalization()
    norm.adapt(first(X_train))
    
    model = Sequential([
        first,
        norm,
        Dense(3),
        Dropout(0.4),
        Dense(3),
        Dense(3),
        Dense(3),
        Dense(3),
        Dense(1)
    ])

    model.compile(loss='mean_absolute_percentage_error', optimizer='adam')
    model.fit(X_train, y_train, validation_data=eval_set, epochs=epochs, batch_size=batch_size, verbose=1)
    
    return model

In [26]:
lstm_model = train_keras(
    LSTM(5, input_shape=(kX_train.shape[1], kX_train.shape[2])),
    X_train = kX_train,
    y_train = ky_train,
    eval_set = (kX_test, ky_test),
    epochs=20
)

Epoch 1/20
415/415 [==============================] - 8s 16ms/step - loss: 93.7758 - val_loss: 60.7979
Epoch 2/20
415/415 [==============================] - 7s 16ms/step - loss: 49.7836 - val_loss: 27.8960
Epoch 3/20
415/415 [==============================] - 7s 16ms/step - loss: 27.0687 - val_loss: 1.9634
Epoch 4/20
415/415 [==============================] - 7s 17ms/step - loss: 0.4921 - val_loss: 1.0716
Epoch 5/20
415/415 [==============================] - 7s 16ms/step - loss: 0.3056 - val_loss: 1.0204
Epoch 6/20
415/415 [==============================] - 7s 16ms/step - loss: 0.3057 - val_loss: 1.0350
Epoch 7/20
415/415 [==============================] - 7s 16ms/step - loss: 0.3061 - val_loss: 1.0598
Epoch 8/20
415/415 [==============================] - 7s 16ms/step - loss: 0.3058 - val_loss: 1.0733
Epoch 9/20
415/415 [==============================] - 7s 17ms/step - loss: 0.3060 - val_loss: 1.0324
Epoch 10/20
415/415 [==============================] - 7s 16ms/step - loss: 0.3060 - v

In [27]:
gru_model = train_keras(
    GRU(10, input_shape=(kX_train.shape[1], kX_train.shape[2])),
    X_train = kX_train,
    y_train = ky_train,
    eval_set = (kX_test, ky_test),
    epochs=15
)

Epoch 1/15
415/415 [==============================] - 11s 24ms/step - loss: 91.7076 - val_loss: 54.1086
Epoch 2/15
415/415 [==============================] - 9s 22ms/step - loss: 48.6972 - val_loss: 26.9493
Epoch 3/15
415/415 [==============================] - 9s 22ms/step - loss: 22.1228 - val_loss: 1.0728
Epoch 4/15
415/415 [==============================] - 9s 22ms/step - loss: 0.3055 - val_loss: 1.0597
Epoch 5/15
415/415 [==============================] - 9s 23ms/step - loss: 0.3055 - val_loss: 1.0421
Epoch 6/15
415/415 [==============================] - 10s 24ms/step - loss: 0.3058 - val_loss: 1.0491
Epoch 7/15
415/415 [==============================] - 9s 23ms/step - loss: 0.3061 - val_loss: 1.0714
Epoch 8/15
415/415 [==============================] - 10s 23ms/step - loss: 0.3063 - val_loss: 1.0344
Epoch 9/15
415/415 [==============================] - 9s 22ms/step - loss: 0.3065 - val_loss: 1.0702
Epoch 10/15
415/415 [==============================] - 9s 22ms/step - loss: 0.3069 

In [30]:
tcn_model = train_keras(
    TCN(nb_filters=4, input_shape=(kX_train.shape[1], kX_train.shape[2])),
    X_train = kX_train,
    y_train = ky_train,
    eval_set = (kX_test, ky_test),
    epochs=10
)

Epoch 1/10
415/415 [==============================] - 51s 119ms/step - loss: 43.3444 - val_loss: 22.9038
Epoch 2/10
415/415 [==============================] - 50s 121ms/step - loss: 32.1650 - val_loss: 11.0125
Epoch 3/10
415/415 [==============================] - 54s 129ms/step - loss: 31.4017 - val_loss: 10.9899
Epoch 4/10
415/415 [==============================] - 50s 119ms/step - loss: 31.2254 - val_loss: 10.9054
Epoch 5/10
415/415 [==============================] - 51s 123ms/step - loss: 30.9823 - val_loss: 10.8909
Epoch 6/10
415/415 [==============================] - 49s 118ms/step - loss: 30.8620 - val_loss: 10.8142
Epoch 7/10
415/415 [==============================] - 62s 149ms/step - loss: 30.5238 - val_loss: 10.7508
Epoch 8/10
415/415 [==============================] - 62s 150ms/step - loss: 30.1874 - val_loss: 10.6000
Epoch 9/10
415/415 [==============================] - 47s 113ms/step - loss: 29.8405 - val_loss: 10.5117
Epoch 10/10
415/415 [==============================] - 